# Load and test network on any input

In [ ]:
import torch
import numpy as np

import poker_env.datatypes as pdt
from poker_env.env import Poker
from poker_env.config import Config
from models.networks import OmahaActor,OmahaQCritic,OmahaObsQCritic,CombinedNet
from models.model_layers import ProcessHandBoard
from models.model_utils import strip_padding,unspool,hardcode_handstrength

In [ ]:
config = Config()
game_object = pdt.Globals.GameTypeDict[pdt.GameTypes.OMAHAHI]

env_params = {
    'game':pdt.GameTypes.OMAHAHI,
    'betsizes': game_object.rule_params['betsizes'],
    'bet_type': game_object.rule_params['bettype'],
    'n_players': 2,
    'pot':1,
    'stacksize': game_object.state_params['stacksize'],
    'cards_per_player': game_object.state_params['cards_per_player'],
    'starting_street': game_object.starting_street,
    'global_mapping':config.global_mapping,
    'state_mapping':config.state_mapping,
    'obs_mapping':config.obs_mapping,
    'shuffle':True
}

env = Poker(env_params)

nS = env.state_space
nA = env.action_space
nB = env.betsize_space
seed = 1235
device = torch.device(cuda_dict[args.gpu] if torch.cuda.is_available() else "cpu")
gpu1 = 'cuda:0'
gpu2 = 'cuda:1'
network_params                                = config.network_params
network_params['device']                      = device

# Instantiate net and copy weights

In [ ]:
def copy_weights(net,path):
    if torch.cuda.is_available():
        layer_weights = torch.load(path)
    else:
        layer_weights = torch.load(path,map_location=torch.device('cpu'))
    for name, param in net.named_parameters():
        if name in layer_weights:
            print('update_weights',name)
            param.data.copy_(layer_weights[name].data)
            param.requires_grad = False

In [ ]:
net = ProcessHandBoard(network_params,hand_length=4)

In [ ]:
copy_weights(net,config.network_params['actor_hand_recognizer_path'])

In [ ]:
for name, param in net.named_parameters():
    print(name,param.requires_grad)

In [ ]:
net_input = torch.tensor([[[13.,  4., 11.,  3.,  5.,  1., 14.,  2., 11.,  2., 14.,  1.,  2.,  3.,
           9.,  2., 13.,  3.],
         [13.,  4., 11.,  3.,  5.,  1., 14.,  2., 11.,  2., 14.,  1.,  2.,  3.,
           9.,  2., 13.,  3.]]])

In [ ]:
net_input.size()

In [ ]:
def compare_weights(net):
    path = '/Users/morgan/Code/PokerAI/poker/checkpoints/frozen_layers/hand_board_weights'
    if torch.cuda.is_available():
        layer_weights = torch.load(path)
    else:
        layer_weights = torch.load(path,map_location=torch.device('cpu'))
    print(net)
    for name, param in net.named_parameters():
        if name in layer_weights:
            print(f'Layer {name},Equal {np.array_equal(param.data.numpy(),layer_weights[name].data.numpy())}')


# compare vs baseline

In [ ]:
compare_weights(net)

In [ ]:
out = net(net_input)

In [ ]:
print(hardcode_handstrength(net_input))

# Duplicate Poker

Where hands are reversed

In [6]:
from collections import deque
import poker_env.datatypes as pdt
from tournament import tournament,generate_duplicate_decks
from poker_env.env import Poker
from poker_env.config import Config
import copy

In [7]:
config = Config()
game_object = pdt.Globals.GameTypeDict[pdt.GameTypes.OMAHAHI]

env_params = {
    'game':pdt.GameTypes.OMAHAHI,
    'betsizes': game_object.rule_params['betsizes'],
    'bet_type': game_object.rule_params['bettype'],
    'n_players': 2,
    'pot':game_object.state_params['pot'],
    'stacksize': game_object.state_params['stacksize'],
    'cards_per_player': game_object.state_params['cards_per_player'],
    'starting_street': game_object.starting_street,
    'global_mapping':config.global_mapping,
    'state_mapping':config.state_mapping,
    'obs_mapping':config.obs_mapping,
    'shuffle':False
}

In [8]:
env = Poker(env_params)

In [9]:
duplicate_decks = generate_duplicate_decks(2)

In [20]:
state,obs,done,action_mask,betsize_mask = env.reset(copy.deepcopy(duplicate_decks[0]))

In [21]:
print(env.players['SB'].hand)
print(env.players['BB'].hand)
print(env.board)
print(env.deck.deck)
print(len(env.deck.deck))

[[7, 2], [11, 3], [10, 3], [4, 2]]
[[2, 2], [4, 4], [3, 1], [7, 1]]
[13, 2, 9, 2, 2, 1, 7, 3, 0, 0]
deque([[5, 1], [6, 4], [14, 4], [9, 1], [2, 3], [11, 4], [5, 2], [6, 3], [3, 4], [3, 3], [13, 3], [7, 4], [8, 2], [9, 4], [10, 4], [8, 4], [11, 2], [5, 4], [12, 2], [12, 1], [12, 3], [4, 3], [6, 1], [6, 2], [2, 4], [4, 1], [10, 2], [9, 3], [13, 1], [8, 1], [10, 1], [8, 3], [14, 1], [14, 3], [13, 4], [14, 2], [12, 4], [5, 3], [3, 2], [11, 1]], maxlen=52)
40


In [24]:
state,obs,done,action_mask,betsize_mask = env.reset()

In [25]:
print(env.players['SB'].hand)
print(env.players['BB'].hand)
print(env.board)
print(env.deck.deck)
print(len(env.deck.deck))

[[14, 4], [14, 3], [14, 2], [14, 1]]
[[13, 4], [13, 3], [13, 2], [13, 1]]
[12, 4, 12, 3, 12, 2, 12, 1, 0, 0]
deque([[2, 1], [2, 2], [2, 3], [2, 4], [3, 1], [3, 2], [3, 3], [3, 4], [4, 1], [4, 2], [4, 3], [4, 4], [5, 1], [5, 2], [5, 3], [5, 4], [6, 1], [6, 2], [6, 3], [6, 4], [7, 1], [7, 2], [7, 3], [7, 4], [8, 1], [8, 2], [8, 3], [8, 4], [9, 1], [9, 2], [9, 3], [9, 4], [10, 1], [10, 2], [10, 3], [10, 4], [11, 1], [11, 2], [11, 3], [11, 4]], maxlen=52)
40


In [12]:
print(len(duplicate_decks))

2


In [13]:
len(duplicate_decks[0])

52

In [14]:
duplicate_decks[0]

deque([[5, 1],
       [6, 4],
       [14, 4],
       [9, 1],
       [2, 3],
       [11, 4],
       [5, 2],
       [6, 3],
       [3, 4],
       [3, 3],
       [13, 3],
       [7, 4],
       [8, 2],
       [9, 4],
       [10, 4],
       [8, 4],
       [11, 2],
       [5, 4],
       [12, 2],
       [12, 1],
       [12, 3],
       [4, 3],
       [6, 1],
       [6, 2],
       [2, 4],
       [4, 1],
       [10, 2],
       [9, 3],
       [13, 1],
       [8, 1],
       [10, 1],
       [8, 3],
       [14, 1],
       [14, 3],
       [13, 4],
       [14, 2],
       [12, 4],
       [5, 3],
       [3, 2],
       [11, 1],
       [7, 3],
       [2, 1],
       [9, 2],
       [13, 2],
       [7, 1],
       [3, 1],
       [4, 4],
       [2, 2],
       [4, 2],
       [10, 3],
       [11, 3],
       [7, 2]])

In [15]:
duplicate_decks[1]

deque([[5, 1],
       [6, 4],
       [14, 4],
       [9, 1],
       [2, 3],
       [11, 4],
       [5, 2],
       [6, 3],
       [3, 4],
       [3, 3],
       [13, 3],
       [7, 4],
       [8, 2],
       [9, 4],
       [10, 4],
       [8, 4],
       [11, 2],
       [5, 4],
       [12, 2],
       [12, 1],
       [12, 3],
       [4, 3],
       [6, 1],
       [6, 2],
       [2, 4],
       [4, 1],
       [10, 2],
       [9, 3],
       [13, 1],
       [8, 1],
       [10, 1],
       [8, 3],
       [14, 1],
       [14, 3],
       [13, 4],
       [14, 2],
       [12, 4],
       [5, 3],
       [3, 2],
       [11, 1],
       [7, 3],
       [2, 1],
       [9, 2],
       [13, 2],
       [4, 2],
       [10, 3],
       [11, 3],
       [7, 2],
       [7, 1],
       [3, 1],
       [4, 4],
       [2, 2]])